In [1]:
"""https://arxiv.org/pdf/1512.03385.pdf"""

'https://arxiv.org/pdf/1512.03385.pdf'

In [2]:
from tensorflow.keras.layers import Conv2D, MaxPool2D,\
BatchNormalization, Dense, Dropout, Activation, Flatten,\
Input, ZeroPadding2D, Add, AveragePooling2D

from tensorflow.keras import Model

import tensorflow as tf

In [3]:
def identity_block(x, filters):
    f1, f2 = filters
    shortcut = x
    
    
    x = Conv2D(f1, kernel_size=(3,3), strides=(1, 1), padding='same', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               bias_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    
    
    x = Conv2D(f2, kernel_size=(3,3), strides=(1, 1), padding='same', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               bias_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x



In [4]:
def conv_block(x, s, filters):
    f1, f2 = filters
    shortcut = x
    
    x = Conv2D(f1, kernel_size=(3,3), strides=(s, s), padding='same', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               bias_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    
    x = Conv2D(f2, kernel_size=(3,3), strides=(1, 1), padding='same', 
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               bias_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = BatchNormalization()(x)

    
    shortcut = Conv2D(f2, kernel_size=(3,3), strides=(s, s), padding='same',
               kernel_regularizer=tf.keras.regularizers.l2(0.001),
               bias_regularizer=tf.keras.regularizers.l2(0.001))(shortcut)
    shortcut = BatchNormalization()(shortcut)

    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)

    return x

In [5]:
def get_model():

    input_im = Input(shape=(224,224,3)) 
    x = ZeroPadding2D(padding=(1,1))(input_im)

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool2D((3, 3), strides=(2, 2))(x)

    
    x = identity_block(x, filters=(64, 64))
    x = identity_block(x, filters=(64, 64))
    x = identity_block(x, filters=(64, 64))


    x = conv_block(x, s=2, filters=(64, 128))
    x = identity_block(x, filters=(128, 128))
    x = identity_block(x, filters=(128, 128))
    x = identity_block(x, filters=(128, 128))

    x = conv_block(x, s=2, filters=(128, 256))
    x = identity_block(x, filters=(256, 256))
    x = identity_block(x, filters=(256, 256))
    x = identity_block(x, filters=(256, 256))
    x = identity_block(x, filters=(256, 256))
    x = identity_block(x, filters=(256, 256))

    x = conv_block(x, s=2, filters=(256, 512))
    x = identity_block(x, filters=(512, 512))
    x = identity_block(x, filters=(512, 512))

    x = AveragePooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    x = Dense(1000, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

    model = Model(inputs=input_im, outputs=x)

    return model

In [8]:
get_model().summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 226, 226, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 110, 110, 64) 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 110, 110, 64) 256         conv2d_36[0][0]                  
____________________________________________________________________________________________